In [1]:
%matplotlib inline
# Import  required module
from StochasticInference import *

# Create an empty StochasticInference object to initiate the inference procedure
spid = StochasticInference()

# Import a model that you want to use for inference
spid.import_sbml('sbml_example.xml')



<libsbml.SBMLDocument; proxy of <Swig Object of type 'SBMLDocument *' at 0x000001CA45F10930> >

In [ ]:

#########################################  Part - I  ######################################### 

# Handling the data

# This is how you can define properties while importing data from a csv file (to filter out the required data)
# Usage - 
# properties_dict = {'Property1': 'Value1','Property2': 'Value2'}
# data = import_timeseries('data.csv',time_column = 1, value_column = 2, properties = properties_dict)

# Example - 
data = import_timeseries('test_data.csv', time_column = 2, value_column = 4, properties = {3 : 51})

# Plot experimental data, give a True value to the optional argument plot_show (uncomment to run)
# import_timeseries('test_data.csv', time_column = 2, value_column = 4, plot_show = True)

import numpy as np
import matplotlib.pyplot as plt  

t_exp = data.get_keys()
timepoints = np.linspace(t_exp[0],t_exp[-1],len(t_exp))
bs_data, m = spid.simulate(timepoints, type = 'deterministic')
simtime = timepoints
simdata = bs_data[:,m.get_species_index('c1')]
# simtime, simdata = model.simulate_roadrunner(timepoints)

# Create artificial data for model to test the code
# SKIP this part you already have your data, store it inside exp_data

lines = []
exp_value = []
for t,d in zip(simtime, simdata):
    v = d + np.random.normal(0,0.5)
    exp_value.append(v)
    lines.append([t,v])
import csv 
with open('test_data_noisy.csv','w') as f:
    writer = csv.writer(f)
    writer.writerows(lines)
    f.close()

plt.plot(simtime, exp_value, label = 'Data')
plt.plot(simtime, simdata, label = 'Model simulation')
plt.legend()
plt.title('Simulated model and artificial (noisy) data')
plt.show()



In [ ]:

#########################################  Part - II  ######################################### 

# Identifiability analysis (NotImplemented yet)


In [ ]:

#########################################  Part - III  ######################################### 

# Parameter identification using MCMC

# Specify the priors for the parameters
spid.prior = {'kc' : [1e-3, 1e3],'k1' : [1e-2, 1e5]}
# Initial guesses for the parameters to estimate
spid.params_to_estimate = {'kc':6, 'k1':1}
fit_model, id_params = spid.run_mcmc(spid.params_to_estimate, spid.prior, timepoints, data,
                                    nwalkers = 8, nsteps = 50, nsamples = 10, measurements = ['c1'], plot_show = False)
# fit_model is with identified parameters' "best" value substituted


In [ ]:


#########################################  Part - IV  ######################################### 

# Post identification analysis (do your own with the results or use this)

res_orig, m_orig = spid.simulate(timepoints, type = 'stochastic')
res, m = spid.simulate(timepoints, type = 'stochastic')

plt.figure()
simtime = timepoints
simdata_orig = res_orig[:,m_orig.get_species_index('c1')]
simdata = res[:,m.get_species_index('c1')]
plt.plot(simtime, exp_value, label ='exp')
plt.plot(simtime, simdata_orig, label = 'original model')
plt.plot(simtime, simdata, label = 'identified model')
plt.legend()
plt.title('Identified model with data')
plt.show()


# Write the fitted model to SBML
fit_model.export_sbml('sbml_fit.xml')